# Making a hybrid CAD CSG model

can be useful for bioshields ...

In [1]:
import openmc
import cadquery as cq
from cad_to_dagmc import CadToDagmc
import dagmc_h5m_file_inspector as di
from pathlib import Path

# Setting the cross section path to the correct location in the docker image.
# If you are running this outside the docker image you will have to change this path to your local cross section path.
openmc.config['cross_sections'] = Path.home() / 'nuclear_data' / 'cross_sections.xml'

In [2]:
major_radius = 1000
minor_radii = 400
thickness = 10

assembly = cq.Assembly()
inner_torus = cq.Solid.makeTorus(major_radius, minor_radii)
outer_torus = cq.Solid.makeTorus(major_radius, minor_radii+thickness)
shell = outer_torus.cut(inner_torus)
assembly.add(shell, name=f"first_wall")
assembly

In [ ]:
dag_mesh = CadToDagmc()
dag_mesh.add_cadquery_object(
    cadquery_object=assembly,
    material_tags="assembly_names",  # Use names as tags
)
dag_mesh.export_dagmc_h5m_file(set_size={'first_wall': 10} )

material_tags found from assembly names: ['first_wall']
Using meshing backend: gmsh
Imprinting assembly for mesh generation
volumes [(3, 1)]
Boundaries for volume 1: [(0, 1), (0, 2)]
Set mesh size 10 for boundary (0, 1)
Set mesh size 10 for boundary (0, 2)
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Circle)
Info    : [  0%] Meshing curve 4 (Circle)
Info    : [  0%] Meshing curve 3 (Circle)
Info    : [  0%] Meshing curve 2 (Circle)
Info    : Done meshing 1D (Wall 0.0095409s, CPU 0.289115s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 2 (Torus, MeshAdapt)
Info    : [  0%] Meshing surface 1 (Torus, MeshAdapt)


In [ ]:
di.convert_h5m_to_vtkhdf(h5m_filename='dagmc.h5m', vtkhdf_filename= 'dagmc.vtkhdf')

'dagmc.vtkhdf'

In [ ]:
dag_universe = openmc.DAGMCUniverse(filename='dagmc.h5m')

In [ ]:
surf_x1 = openmc.XPlane(x0=-1000.0, name='x1')
surf_x2 = openmc.XPlane(x0=1000.0, name='x2')
surf_y1 = openmc.YPlane(y0=-1000.0, name='y1')
surf_y2 = openmc.YPlane(y0=1000.0, name='y2')
surf_z1 = openmc.ZPlane(z0=-1000.0, name='z1')
surf_z2 = openmc.ZPlane(z0=1000.0, name='z2')

region_inner_bioshield = +surf_x1 & -surf_x2 & +surf_y1 & -surf_y2 & +surf_z1 & -surf_z2
inner_bioshield = openmc.Cell(name='inner bioshield', region=region_inner_bioshield, fill=dag_universe)


: 

In [ ]:
inner_bioshield.plot()